# 1. Read in all files i.e. data ingestion

Basic csv reading code added. Next: use llamaparse's own data loaders

In [2]:
# connecting to google drive drive:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# you can get all related files and csv's from here:

https://drive.google.com/drive/folders/1a_6BKzeR9W9hDqXYUApfDPPof1w7pzMG

In [4]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os

# The CSV download link
path = '/content/drive/MyDrive/Omdena_Challenge/new_csvs/new_LK_tea_dataset_updated.csv'

# Read the Google Sheet into a pandas DataFrame
all_data = pd.read_csv(path)
all_data.fillna('', inplace=True)
# Display the DataFrame
all_data.head()


,id,class,filename,path,url,language_label,required_ocr,has_tables,text_path,quality,data_origin,retrieved_date_of_issuance,issuing_authority,retrieved_topic,text_content,PDF_or_text
0,0,ACT,ACT_1.txt,,https://www.lawnet.gov.lk/tea-subsidy-3/,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Subsidy – LawNet,"Tea Subsidy\nActs Nos. 12 of 1958, 66 of 1961....",Text
1,1,ACT,ACT_2.txt,,https://www.lawnet.gov.lk/tea-small-holdings-d...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Small Holdings Development (Amendment) – L...,|\n\n**Tea Small Holdings Development (Amendme...,Text
2,2,ACT,ACT_3.txt,,https://www.lawnet.gov.lk/tea-small-holdings-d...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Small Holdings Development (Amendment) – L...,|\n\n**Tea Small Holdings Development (Amendme...,Text
3,3,ACT,ACT_4.txt,,https://www.lawnet.gov.lk/tea-small-holdings-d...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Small Holdings Development Law – LawNet,Tea Small Holdings Development Law\nA LAW TO E...,Text
4,4,ACT,ACT_5.txt,,https://www.lawnet.gov.lk/tea-research-board-a...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Research Board (Amendment) – LawNet,|\n\n**Tea Research Board (Amendment)** |\n\n|...,Text


In [5]:
# Display the loaded data to check for Tea Control Act which is longer than 50,000 characters
all_data[all_data['retrieved_topic']=='Tea Control – LawNet']

,id,class,filename,path,url,language_label,required_ocr,has_tables,text_path,quality,data_origin,retrieved_date_of_issuance,issuing_authority,retrieved_topic,text_content,PDF_or_text
19,19,ACT,ACT_20.txt,,https://www.lawnet.gov.lk/tea-control-7/,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Control – LawNet,"""Tea Control\nActs Nos. 51 of 1957. 26 of 1962...",Text


In [ ]:
import pandas as pd

# Function to count words and characters in text
def count_words_and_characters(text):
    words = text.split()  # Split by whitespace to get words
    num_words = len(words)
    num_characters = len(text)
    return num_words, num_characters

# Function to count words and characters for the 'text_content' column in the DataFrame
def count_stats_in_dataframe(df):
    total_words = 0
    total_characters = 0

    # Iterate over the 'text_content' column in the DataFrame
    for text in df['text_content'].dropna():  # Drop any NaN values in text_content
        num_words, num_characters = count_words_and_characters(text)
        total_words += num_words
        total_characters += num_characters

    return total_words, total_characters

# Main script
if __name__ == "__main__":

    # Count words and characters
    total_words, total_characters = count_stats_in_dataframe(all_data)

    # Output the results
    print("\nTotal Stats for 'text_content' column in DataFrame:")
    print(f"Total Words: {total_words}")
    print(f"Total Characters: {total_characters}")



Total Stats for 'text_content' column in DataFrame:
Total Words: 186342
Total Characters: 1120726


# 2. Feed all the PDF files as well as ACTs to llamaparse and get parsed chunks in markdown format

for 173 files, around 600 chunks are returned

* @TODO convert ACTs/text via Fast mode (1 cred. / 3 p)
*@TODO convert PDFs via Accurate mode (1 cred. / 1 p.),
* and don't use continuous mode (10 credits)...except for PDFs with Tables....becuase if they span multiple pages, it would be an issue.....................67 files have tables but not all span multiple pages

In [ ]:
!pip --quiet install llama-index-core llama-parse llama-index-readers-file python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00


In [ ]:
import os
import getpass
os.environ['LLAMA_CLOUD_API_KEY'] = getpass.getpass('Enter your LLamacloud API Key: ')
# os.environ['GROQ_API_KEY'] = getpass.getpass('Enter your GROQ API Key: ')

Enter your LLamacloud API Key: ··········


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# my version



import pandas as pd
from dotenv import load_dotenv
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# Load environment variables (assuming LLAMA_CLOUD_API_KEY is needed somewhere)
load_dotenv()

# Assuming you have your DataFrame `all_data` already loaded
# Example: all_data = pd.read_csv("your_data.csv") or whatever source you're using

# Prepare LlamaParse instance for PDF file that uses accurate mode which is true by default:
# it hallucinates so changing prompt

# pdf_parser = LlamaParse(result_type='text', verbose=True, continuous_mode=True, language= 'en',
#                     parsing_instruction = "You are parsing a regulatory document related to the Sri Lankan Tea Industry. Please give me a json with the title and date of issuance of the document"
#                     )  # "markdown" and "text" are available


# also doesn't work, don't use parsing prompt
pdf_parser = LlamaParse(result_type='markdown', num_workers=8)  # "markdown" and "text" are available

# Prepare LlamaParse instance for text file that uses fast mode:
text_parser = LlamaParse(result_type='markdown', Set_fast_mode = True, num_workers=8)  # "markdown" and "text" are available

# set workers =8 makes it 'fake' parallel

# default is text, modified to markdown for next time. Also, changed number of workers to 8 for next time


In [ ]:
# # # works perfectly but no metadata such as filename or path returned which I need later to consolidate results with all_data,
# # opened an issue here: https://github.com/run-llama/llama_parse/issues/493
# # trying with simpledirectoryloader in next cell:

# import pandas as pd
# from llama_parse import LlamaParse
# import asyncio

# # Initialize parsers
# pdf_parser = LlamaParse(result_type='markdown', num_workers=8)
# pdf_paths = all_data[all_data['PDF_or_text'] == 'PDF']['path'].tolist()
# pdf_documents = await pdf_parser.aload_data(pdf_paths)

# text_parser = LlamaParse(result_type='markdown', Set_fast_mode=True, num_workers=8)
# text_paths = all_data[all_data['PDF_or_text'] == 'Text']['text_path'].tolist()
# text_documents = await text_parser.aload_data(text_paths)


# all_documents = pdf_documents + text_documents


In [ ]:
# # WRONG WAY TO DO ASYNC, use in-built functions like in next cell

# import pandas as pd
# from llama_parse import LlamaParse
# import asyncio

# # Initialize parsers
# pdf_parser = LlamaParse(result_type='markdown', num_workers=8)
# text_parser = LlamaParse(result_type='markdown', Set_fast_mode=True, num_workers=8)


# # Async function to process PDF documents
# async def process_pdf_documents(pdf_paths):
#     if pdf_paths:  # Only process if there are PDF paths
#         file_extractor = {".pdf": pdf_parser}
#         pdf_documents = SimpleDirectoryReader(input_files=pdf_paths, file_extractor=file_extractor).load_data()

#         return pdf_documents
#     return []

# # Async function to process Text documents
# async def process_text_documents(text_paths):
#     if text_paths:  # Only process if there are text paths
#         file_extractor = {".txt": text_parser}
#         text_documents = SimpleDirectoryReader(input_files=text_paths, file_extractor=file_extractor).load_data()
#         return text_documents
#     return []

# # Async function to process the DataFrame and save to CSV
# async def process_and_save_df(all_data):
#     # Collect file paths for PDFs and Text documents
#     pdf_paths = all_data[all_data['PDF_or_text'] == 'PDF']['path'].tolist()
#     text_paths = all_data[all_data['PDF_or_text'] == 'Text']['text_path'].tolist()

#     # Process PDFs and Texts concurrently
#     pdf_documents = await process_pdf_documents(pdf_paths)
#     text_documents = await process_text_documents(text_paths)

#     # Combine the results
#     all_documents = pdf_documents + text_documents

#     if all_documents:
#         return all_documents
#     else:
#         print("No documents found for processing.")
#         return []

# # Example usage
# async def main():
#     # Assuming `all_data` is a DataFrame that includes columns like 'PDF_or_text', 'path', 'text_path'
#     # all_data = pd.DataFrame({
#     #     'PDF_or_text': ['PDF', 'Text', 'PDF', 'Text'],
#     #     'path': ['./data/file1.pdf', '', './data/file3.pdf', ''],
#     #     'text_path': ['', './data/file2.txt', '', './data/file4.txt']
#     # })

#     # Process documents and get the list of processed documents
#     processed_documents = await process_and_save_df(all_data)

#     if processed_documents:
#         return processed_documents
#     else:
#         print("No documents to process.")

# # Run the asynchronous main function
# processed_documents = asyncio.run(main())


Started parsing the file under job_id 7b26bab3-71a1-45e9-a4bf-903b1ac4d4b5
Started parsing the file under job_id 29961bcb-92d6-400b-a587-fab5904e0178
..Started parsing the file under job_id 974f8975-9200-4726-8e6f-c617a33cd4fd
Started parsing the file under job_id 17f3c924-106a-46a2-9817-26efe7aaf37b
.Started parsing the file under job_id 64de5715-0129-450e-a239-d872c4a9b09a
.Started parsing the file under job_id a95c82df-ca33-4513-9e98-8a8dab66f710
.Started parsing the file under job_id 3e0d3e94-c7cd-482c-abe3-70bafe4b5351
Started parsing the file under job_id 16931b37-20c7-48f6-bc12-5b445922b88c
.Started parsing the file under job_id ac10660b-d03b-4cce-ac43-182e2e16eb8d
Started parsing the file under job_id cd8253b1-8b7d-4ec4-a46e-2360dc1f0c4d
Started parsing the file under job_id 6b408812-8fd4-4bd8-b4e2-fd83892b7589
.Started parsing the file under job_id 430be052-b49f-4e7e-aed1-b2eff1c1836f
Started parsing the file under job_id 2c2df3a1-5673-4d45-8ef6-bddf2377e9de
Started parsing th

In [ ]:
# async using inbuilt aload() function with simpledirectory loader. RunLLM AI assitant on llamaindex website told me this. For some reason, there is no example of simpledirectoryreader
# with aload, both are treated as separate, however, non-async takes hours, this took under 15 minutes toget parsed results for all documents


import pandas as pd
from llama_parse import LlamaParse
import asyncio

# Initialize parsers
pdf_parser = LlamaParse(result_type='markdown', num_workers=8)
text_parser = LlamaParse(result_type='markdown', Set_fast_mode=True, num_workers=8)



pdf_paths = all_data[all_data['PDF_or_text'] == 'PDF']['path'].tolist()
pdf_file_extractor = {".pdf": pdf_parser}
pdf_reader = SimpleDirectoryReader(input_files=pdf_paths, file_extractor=pdf_file_extractor)
pdf_documents = await pdf_reader.aload_data()

text_paths = all_data[all_data['PDF_or_text'] == 'Text']['text_path'].tolist()
text_file_extractor = {".txt": text_parser}
text_reader = SimpleDirectoryReader(input_files=text_paths, file_extractor=text_file_extractor)
text_documents = await text_reader.aload_data()


# Combine the results
processed_documents = pdf_documents + text_documents

Started parsing the file under job_id aae7b330-61e5-4fc4-84cf-55751c5d3c75
Started parsing the file under job_id c2e3ff9d-57fb-4eee-9491-ddd7df635b88
Started parsing the file under job_id d93baee9-b91f-407d-83fb-421da71dd264
Started parsing the file under job_id ee0bf606-1a9a-43bc-a5d1-62b3d86e0339
Started parsing the file under job_id 00f39d35-544d-4ade-bbb0-26ba21ce7f5c
Started parsing the file under job_id cceb5948-b3ee-467a-b146-b8700f20ef18
Started parsing the file under job_id b2e725b9-cc55-4ba3-b65f-68995431d816
Started parsing the file under job_id e31a2499-51f2-401c-a91a-43281aa6a76c
Started parsing the file under job_id d171b370-2e95-4307-b10d-62042a805fdc
Started parsing the file under job_id 15e34b12-bb55-43d6-8819-380ff84fe43b
Started parsing the file under job_id c881e4f4-3201-464c-9f80-7393f08e6012
Started parsing the file under job_id 5e70ce2c-8318-4d53-9bad-f83f4afb5ab0
Started parsing the file under job_id 08277527-4e00-4be9-b28e-09241a73a03a
Started parsing the file 

In [ ]:
processed_documents[0]

Document(id_='5373cab0-f60b-46a1-a2cd-f13b79b77050', embedding=None, metadata={'file_path': '/content/drive/MyDrive/Omdena_Challenge/new_LK_tea_dataset/Circulers/07_General-Instruction-for-the-Submission-of-Tea-Samples-for-Testing_compressed.pdf', 'file_name': '07_General-Instruction-for-the-Submission-of-Tea-Samples-for-Testing_compressed.pdf', 'file_type': 'application/pdf', 'file_size': 14710, 'creation_date': '2024-11-18', 'last_modified_date': '2024-11-14'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\n\n# GENERAL INSTRUCTION FOR THE SUBMISSION OF TEA SAMPLES FOR TESTING\n\n# 1. Submission of tea sample\n\n- Sample of tea should be submitte

Attributes to Extract:

id_: Unique identifier for the document.
embedding: An optional field, which may be None if not set.
metadata: A dictionary containing any associated metadata (in this case, it's empty).
relationships: A dictionary containing any relationships (empty in this case).
text: The content of the document itself.
mimetype: The MIME type of the document (in this case, it's 'text/plain').
start_char_idx, end_char_idx: These are None in your example, but may hold character index information if applicable.

Optional:

metadata_template, text_template, and other fields can be ignored if not needed.

In [ ]:
len(processed_documents)

607

In [ ]:
type(processed_documents)

list

In [ ]:
type(processed_documents[0])

llama_index.core.schema.Document

In [ ]:
import pandas as pd
import json
from llama_index.core.schema import Document
import csv

# Sample function to convert a single Document to a dictionary
def document_to_dict(document: Document):
    doc_dict = {
        "id": document.id_,
        "embedding": document.embedding,  # May be None
        "metadata": document.metadata,  # Empty in your case
        "excluded_embed_metadata_keys": document.excluded_embed_metadata_keys,
        "excluded_llm_metadata_keys": document.excluded_llm_metadata_keys,
        "relationships": document.relationships,  # Empty in your case
        "text": document.text,  # The markdown content
        "mimetype": document.mimetype,
        "start_char_idx": document.start_char_idx,
        "end_char_idx": document.end_char_idx,
        "text_template": document.text_template,
        "metadata_template": document.metadata_template,
        "metadata_seperator": document.metadata_seperator
    }
    return doc_dict

# Convert the list of Document objects to a list of dictionaries
def documents_to_dicts(documents: list):
    return [document_to_dict(doc) for doc in documents]

# Convert the list of dictionaries to a pandas DataFrame
def documents_to_dataframe(documents: list):
    # Convert documents list to dicts
    doc_dicts = documents_to_dicts(documents)

    # Convert list of dictionaries to a DataFrame
    df = pd.DataFrame(doc_dicts)
    return df


processed_df = documents_to_dataframe(processed_documents)


# Show the resulting DataFrame (markdown text will be preserved)
processed_df




,id,embedding,metadata,excluded_embed_metadata_keys,excluded_llm_metadata_keys,relationships,text,mimetype,start_char_idx,end_char_idx,text_template,metadata_template,metadata_seperator
0,5373cab0-f60b-46a1-a2cd-f13b79b77050,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
1,0aeab14d-dfd4-4ab2-b2bd-2c1226a5eec4,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
2,da3be640-2bc8-4ee1-8d64-62ba823befa8,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},05. Any further clarification in this regard c...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
3,7feecbd4-47af-4c52-948c-12a7b898a386,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# Sri Lanka Tea Board Minimum Quality Standard...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
4,125a38e8-5901-49b9-be59-e28d56a8caee,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# Sri Lanka Tea Board Minimum Quality Standard...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,85446648-b22d-44e4-99da-734d0991b3a9,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# Definitions\n\n” Minister ” means the Minist...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
603,1e84f409-7189-4a0c-b52d-f689d7ae12dd,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# PARLIAMENT OF THE DEMOCRATIC SOCIALIST REPUB...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
604,7d37d11e-5fc6-4db9-860d-2207384bd193,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},remuneration as may be specified in the releva...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
605,2f98a7ce-4991-4a9c-a05e-e378540ea6da,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# PARLIAMENT OF THE DEMOCRATIC SOCIALIST REPUB...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n


In [ ]:
# Sample function to save the DataFrame to CSV with markdown preservation
def save_dataframe_to_csv(df, filename):
    df.to_csv(filename,
              index=False,  # Do not write row indices
              quoting=csv.QUOTE_MINIMAL,  # Quote fields that contain special characters
              quotechar='"',  # Use double quotes to wrap fields
              escapechar='\\',  # Escape any quotes inside markdown content
              lineterminator='\n',  # Make sure to preserve newlines in markdown content
              encoding='utf-8')  # Ensure proper encoding to handle special characters in markdown



# Save the DataFrame to a CSV file with markdown content preserved
save_dataframe_to_csv(processed_df, '/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/llamaparse/llama_processed_documents_markdown.csv')


# Extract metadata


In [ ]:
# Read the DataFrame (assuming the 6th column contains dictionaries with the 'text' key)
processed_df = pd.read_csv('/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/llamaparse/llama_processed_documents_markdown.csv')
processed_df

,id,embedding,metadata,excluded_embed_metadata_keys,excluded_llm_metadata_keys,relationships,text,mimetype,start_char_idx,end_char_idx,text_template,metadata_template,metadata_seperator
0,5373cab0-f60b-46a1-a2cd-f13b79b77050,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
1,0aeab14d-dfd4-4ab2-b2bd-2c1226a5eec4,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},# MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
2,da3be640-2bc8-4ee1-8d64-62ba823befa8,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},05. Any further clarification in this regard c...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
3,7feecbd4-47af-4c52-948c-12a7b898a386,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},# Sri Lanka Tea Board Minimum Quality Standard...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
4,125a38e8-5901-49b9-be59-e28d56a8caee,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},# Sri Lanka Tea Board Minimum Quality Standard...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,85446648-b22d-44e4-99da-734d0991b3a9,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},# Definitions\n\n” Minister ” means the Minist...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
603,1e84f409-7189-4a0c-b52d-f689d7ae12dd,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},# PARLIAMENT OF THE DEMOCRATIC SOCIALIST REPUB...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
604,7d37d11e-5fc6-4db9-860d-2207384bd193,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},remuneration as may be specified in the releva...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
605,2f98a7ce-4991-4a9c-a05e-e378540ea6da,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},# PARLIAMENT OF THE DEMOCRATIC SOCIALIST REPUB...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n


In [ ]:
import ast
# Convert the 'metadata' column from string representation to dictionary
processed_df['metadata'] = processed_df['metadata'].apply(ast.literal_eval)

# Now you can access dictionary keys
first_file_name = processed_df['metadata'].iloc[0]['file_name']
print(first_file_name)  # Output: file1.txt


07_General-Instruction-for-the-Submission-of-Tea-Samples-for-Testing_compressed.pdf


In [ ]:
import pandas as pd

# Creating the DataFrame with 'metadata' column containing dictionaries
df_metadata = pd.DataFrame()
# Extracting the 'filename' from the 'metadata' column
df_metadata['file_name'] = processed_df['metadata'].apply(lambda x: x.get('file_name', None))
df_metadata['markdown_content'] = processed_df['text']

# Display the DataFrame
print(df_metadata)


                                             file_name  \
0    07_General-Instruction-for-the-Submission-of-T...   
1    01_Sri-Lanka-Tea-Board-Standards-Directives-fo...   
2    01_Sri-Lanka-Tea-Board-Standards-Directives-fo...   
3    01_Sri-Lanka-Tea-Board-Standards-Directives-fo...   
4    01_Sri-Lanka-Tea-Board-Standards-Directives-fo...   
..                                                 ...   
602                                         ACT_27.txt   
603                                         ACT_28.txt   
604                                         ACT_28.txt   
605                                         ACT_29.txt   
606                                         ACT_29.txt   

                                      markdown_content  
0    # ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...  
1    # MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...  
2    05. Any further clarification in this regard c...  
3    # Sri Lanka Tea Board Minimum Quality Standard...  
4    # Sri Lanka T

# get full markdown content, however for metadata extraction, only pick first and last two chunks

In [ ]:
import pandas as pd

# Group by 'file_name' and apply a custom function to each group
def merge_markdown_content(group):
    # Combine all markdown content in the group with line breaks
    combined_markdown_all = "\n".join(group['markdown_content'])

    # Take the first row's markdown content
    first_row = group.iloc[0]

    # Take the last two rows (if there are at least two)
    last_two_rows = group.tail(2)

    # Combine the first row text_content with the last two rows text_content
    combined_markdown_first_last = first_row['markdown_content'] + " " + " ".join(last_two_rows['markdown_content'])

    # Return a new series with 'file_name' and the combined markdown content
    return pd.Series({
        'file_name': first_row['file_name'],  # Keeping 'file_name' from the first row
        'markdown_content_long': combined_markdown_all,
        'markdown_content_short': combined_markdown_first_last
    })

# Apply the function to each group
# To avoid deprecation warning, use `as_index=True` (default) and reset index after `apply`
merged_df = df_metadata.groupby('file_name', as_index=False).apply(merge_markdown_content)

# Drop the multi-index that `apply` creates (this is important to avoid a hierarchical index)
merged_df = merged_df.reset_index(drop=True)
update_markdown= merged_df.copy()
# View the merged DataFrame
merged_df


<ipython-input-21-e0f3c949a729>:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = df_metadata.groupby('file_name', as_index=False).apply(merge_markdown_content)


,file_name,markdown_content_long,markdown_content_short
0,01_Sri-Lanka-Tea-Board-Standards-Directives-fo...,# MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...,# MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...
1,07_General-Instruction-for-the-Submission-of-T...,# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...,# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...
2,1998 10 30.pdf,"# Sri Lanka Tea Board\n\n574, Galle Road, Colo...","# Sri Lanka Tea Board\n\n574, Galle Road, Colo..."
3,2158-03_E.pdf,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...
4,2245-29_E.pdf,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...
...,...,...,...
162,al-mqs-08 2008 04 24.pdf,# SRI LANKA TEA BOARD\n\n# Circular No.: AL/MQ...,# SRI LANKA TEA BOARD\n\n# Circular No.: AL/MQ...
163,guidelines_on_covid-19_in_tea_plantations_eng_...,# TEA RESEARCH INSTITUTE OF SRI LANKA\n\nIssue...,# TEA RESEARCH INSTITUTE OF SRI LANKA\n\nIssue...
164,mf bl 119.pdf,# Sri Lanka Tea Board\n\n# Tea Commissioner’s ...,# Sri Lanka Tea Board\n\n# Tea Commissioner’s ...
165,scan0002.pdf,# TKANSLATION\n\n# REGISTERED POST\n\n# SRI LA...,# TKANSLATION\n\n# REGISTERED POST\n\n# SRI LA...


# Get metadata via groq

In [ ]:
!pip -q install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
import os
import getpass
# os.environ['LLAMA_CLOUD_API_KEY'] = getpass.getpass('Enter your LLamacloud API Key: ')


os.environ['GROQ_API_KEY'] = getpass.getpass('Enter your GROQ API Key: ')
groq_api=os.environ['GROQ_API_KEY']

Enter your GROQ API Key: ··········


In [ ]:
import pandas as pd
import json
from groq import Groq
from google.colab import userdata
import os
import re

# Assuming Groq API key is set up properly


client = Groq(api_key=groq_api)

def clean_text_for_metadata_extraction(text):
    """
    Clean and preprocess the extracted text content to make it suitable for metadata extraction.
    This will remove unwanted text such as 'Copyright' and any excessive spaces or line breaks.
    """
    if not isinstance(text, str):
        return None

    # Remove specific unwanted text like 'Copyright' (case-insensitive) and everything after it
    text = re.sub(r'copyright[\s\S]*?(\n|\r|\Z)', '', text, flags=re.IGNORECASE)  # Remove copyright section and text after it

    return text

def extract_metadata(row):
    """
    Extract title and published date from the document's text using the Groq API.
    The prompt is designed to instruct the Groq model to return a JSON object containing these two pieces of information.
    """
    text =  row['markdown_content_short']

    # Clean and preprocess the text before passing to the Groq API
    cleaned_text = clean_text_for_metadata_extraction(text)
    if not cleaned_text:
        return None

    prompt = (
        "Please process the following document and only extract the following details in English, "
        "in valid JSON format:\n\n"
        "- parsed_title: A single main title of the document (if available, otherwise return 'None').\n"
        "- parsed_issue_date: A single main published date of the document (if available, otherwise return 'None').\n"
        "- parsed_reference_number: The reference number of the document (if available, otherwise return 'None').\n\n"
        "Document:\n\n"
        f"{cleaned_text}\n\n"
        "Example output format:\n"
        '{"parsed_title": "<extracted title or None>", "parsed_issue_date": "<extracted date or None>", "parsed_reference_number": "<extracted reference number or None>"}\n'
    )


    # Send the prompt to the Groq API
    try:
        completion = client.chat.completions.create(
            # model="llama3-70b-8192",
            # model="llama3-8b-8192",
            model = "llama-3.1-8b-instant", # Make sure this model is appropriate for your task
            messages=[
                {"role": "user", "content": prompt},
                {"role": "assistant", "content": "```json"}
            ],
            stop="```",
        )

        # Try to extract the JSON response
        return completion.choices[0].message.content.strip("```json").strip()

    except (AttributeError, IndexError) as e:
        print(f"Error processing row: {e}")
        return None

# Apply the metadata extraction to the DataFrame
merged_df['metadata_returned'] = merged_df.apply(extract_metadata, axis = 1)


merged_df

,file_name,markdown_content_long,markdown_content_short,metadata_returned
0,01_Sri-Lanka-Tea-Board-Standards-Directives-fo...,# MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...,# MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...,"{\n ""parsed_title"": ""Sri Lanka Tea Board Mini..."
1,07_General-Instruction-for-the-Submission-of-T...,# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...,# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...,"{\n ""parsed_title"": ""GENERAL INSTRUCTION FOR ..."
2,1998 10 30.pdf,"# Sri Lanka Tea Board\n\n574, Galle Road, Colo...","# Sri Lanka Tea Board\n\n574, Galle Road, Colo...","{\n ""parsed_title"": ""ALL REGISTERED MANUFACTU..."
3,2158-03_E.pdf,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,"{\n ""parsed_title"": ""CONSTITUTION OF THE DEMO..."
4,2245-29_E.pdf,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,"{\n ""parsed_title"": ""REGULATIONS made by the ..."
...,...,...,...,...
162,al-mqs-08 2008 04 24.pdf,# SRI LANKA TEA BOARD\n\n# Circular No.: AL/MQ...,# SRI LANKA TEA BOARD\n\n# Circular No.: AL/MQ...,"{\n ""parsed_title"": ""SRI LANKA TEA BOARD"",\n ..."
163,guidelines_on_covid-19_in_tea_plantations_eng_...,# TEA RESEARCH INSTITUTE OF SRI LANKA\n\nIssue...,# TEA RESEARCH INSTITUTE OF SRI LANKA\n\nIssue...,"{\n ""parsed_title"": ""# GUIDELINES ON PREVENTI..."
164,mf bl 119.pdf,# Sri Lanka Tea Board\n\n# Tea Commissioner’s ...,# Sri Lanka Tea Board\n\n# Tea Commissioner’s ...,"{\n ""parsed_title"": ""DELAY IN FURNISHING MONT..."
165,scan0002.pdf,# TKANSLATION\n\n# REGISTERED POST\n\n# SRI LA...,# TKANSLATION\n\n# REGISTERED POST\n\n# SRI LA...,"{\n ""parsed_title"": ""INTRODUCTION OF POISONOU..."


In [ ]:
# Sample function to save the DataFrame to CSV with markdown preservation
def save_dataframe_to_csv(df, filename):
    df.to_csv(filename,
              index=False,  # Do not write row indices
              quoting=csv.QUOTE_MINIMAL,  # Quote fields that contain special characters
              quotechar='"',  # Use double quotes to wrap fields
              escapechar='\\',  # Escape any quotes inside markdown content
              lineterminator='\n',  # Make sure to preserve newlines in markdown content
              encoding='utf-8')  # Ensure proper encoding to handle special characters in markdown



# Save the DataFrame to a CSV file with markdown content preserved
save_dataframe_to_csv(merged_df, '/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/llamaparse/llama_parsed_metadata_instant.csv')


In [ ]:
## just a local backup copy becuase it can take hours to get results (30 min in this case)
bu_merged_df = merged_df.copy()

In [ ]:
merged_df = bu_merged_df.copy()

In [ ]:
# or read from from csv
merged_df = pd.read_csv('/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/llamaparse/llama_parsed_metadata_instant.csv')
merged_df

,file_name,markdown_content_long,markdown_content_short,metadata_returned
0,01_Sri-Lanka-Tea-Board-Standards-Directives-fo...,# MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...,# MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...,"{\n ""parsed_title"": ""Sri Lanka Tea Board Mini..."
1,07_General-Instruction-for-the-Submission-of-T...,# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...,# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...,"{\n ""parsed_title"": ""GENERAL INSTRUCTION FOR ..."
2,1998 10 30.pdf,"# Sri Lanka Tea Board\n\n574, Galle Road, Colo...","# Sri Lanka Tea Board\n\n574, Galle Road, Colo...","{\n ""parsed_title"": ""ALL REGISTERED MANUFACTU..."
3,2158-03_E.pdf,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,"{\n ""parsed_title"": ""CONSTITUTION OF THE DEMO..."
4,2245-29_E.pdf,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,"{\n ""parsed_title"": ""REGULATIONS made by the ..."
...,...,...,...,...
162,al-mqs-08 2008 04 24.pdf,# SRI LANKA TEA BOARD\n\n# Circular No.: AL/MQ...,# SRI LANKA TEA BOARD\n\n# Circular No.: AL/MQ...,"{\n ""parsed_title"": ""SRI LANKA TEA BOARD"",\n ..."
163,guidelines_on_covid-19_in_tea_plantations_eng_...,# TEA RESEARCH INSTITUTE OF SRI LANKA\n\nIssue...,# TEA RESEARCH INSTITUTE OF SRI LANKA\n\nIssue...,"{\n ""parsed_title"": ""# GUIDELINES ON PREVENTI..."
164,mf bl 119.pdf,# Sri Lanka Tea Board\n\n# Tea Commissioner’s ...,# Sri Lanka Tea Board\n\n# Tea Commissioner’s ...,"{\n ""parsed_title"": ""DELAY IN FURNISHING MONT..."
165,scan0002.pdf,# TKANSLATION\n\n# REGISTERED POST\n\n# SRI LA...,# TKANSLATION\n\n# REGISTERED POST\n\n# SRI LA...,"{\n ""parsed_title"": ""INTRODUCTION OF POISONOU..."


In [ ]:
# Using iloc to get the first row's metadata column value
metadata_test_string = merged_df.iloc[0]['metadata_returned']
metadata_test_string

'{\n  "parsed_title": "Sri Lanka Tea Board Minimum Quality Standards for Tea",\n  "parsed_issue_date": "1st January, 2021",\n  "parsed_reference_number": "Circular No: AL/MQS-02/2021"\n}'

In [ ]:
 ## for llama 70b
 parsed_dict = ast.literal_eval(metadata_test_string)
 parsed_dict

{'parsed_title': 'Sri Lanka Tea Board Minimum Quality Standards for Tea',
 'parsed_issue_date': '1st January, 2021',
 'parsed_reference_number': 'Circular No: AL/MQS-02/2021'}

In [ ]:

import pandas as pd
import json

# Sample function to clean and parse metadata string
def parse_json_from_string(metadata_string):
    try:
        parsed_dict = ast.literal_eval(metadata_string)
        return parsed_dict  # Return the parsed JSON object
    except Exception as e:
        print(f"Error processing string as dict: {e}")
        return None

# Assuming your DataFrame is called df and the metadata is in a column called 'metadata'

# Apply the parsing function to the 'metadata' column
merged_df['parsed_metadata'] = merged_df['metadata_returned'].apply(parse_json_from_string)

# # Convert the parsed JSON string into actual JSON objects (list of dictionaries)
# # merged_df['parsed_metadata'] = merged_df['parsed_metadata'].apply(json.loads)

merged_df = merged_df.join(merged_df['parsed_metadata'].apply(pd.Series))


# # # Expand the JSON objects into separate columns
# expanded_columns = merged_df['parsed_metadata'].apply(pd.Series)

# # ideally the above line should work as is and expand column in place, but until pd.concat isn't used it doesn't show up
# # even though it give duplicate metadata columns

# # # Merge the expanded columns back into the original DataFrame
# merged_df = pd.concat([merged_df, expanded_columns], axis=1)

# # # Drop the original 'parsed_metadata' column if it's no longer needed, and also drop duplicate columns
# merged_df.drop(columns=['parsed_metadata', 'metadata_returned' ], inplace=True)

# Display the updated DataFrame
merged_df


Error processing string as dict: malformed node or string on line 4: <ast.Name object at 0x7b932c126b00>
Error processing string as dict: malformed node or string on line 3: <ast.Name object at 0x7b932d8765c0>
Error processing string as dict: '{' was never closed (<unknown>, line 1)
Error processing string as dict: malformed node or string on line 3: <ast.Name object at 0x7b932c1254e0>
Error processing string as dict: '{' was never closed (<unknown>, line 1)


,file_name,markdown_content_long,markdown_content_short,metadata_returned,parsed_metadata,parsed_title,parsed_issue_date,parsed_reference_number
0,01_Sri-Lanka-Tea-Board-Standards-Directives-fo...,# MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...,# MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...,"{\n ""parsed_title"": ""Sri Lanka Tea Board Mini...",{'parsed_title': 'Sri Lanka Tea Board Minimum ...,Sri Lanka Tea Board Minimum Quality Standards ...,"1st January, 2021",Circular No: AL/MQS-02/2021
1,07_General-Instruction-for-the-Submission-of-T...,# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...,# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...,"{\n ""parsed_title"": ""GENERAL INSTRUCTION FOR ...",{'parsed_title': 'GENERAL INSTRUCTION FOR THE ...,GENERAL INSTRUCTION FOR THE SUBMISSION OF TEA ...,24.06.2013,AL/GI
2,1998 10 30.pdf,"# Sri Lanka Tea Board\n\n574, Galle Road, Colo...","# Sri Lanka Tea Board\n\n574, Galle Road, Colo...","{\n ""parsed_title"": ""ALL REGISTERED MANUFACTU...",{'parsed_title': 'ALL REGISTERED MANUFACTURERS...,ALL REGISTERED MANUFACTURERS OF TEA,30th October 1998,Circular of even number dated 22nd November 1993
3,2158-03_E.pdf,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,"{\n ""parsed_title"": ""CONSTITUTION OF THE DEMO...",{'parsed_title': 'CONSTITUTION OF THE DEMOCRAT...,CONSTITUTION OF THE DEMOCRATIC SOCIALIST REPUB...,"09th January, 2020","No. 2158/3, 01- 649"
4,2245-29_E.pdf,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,"{\n ""parsed_title"": ""REGULATIONS made by the ...",{'parsed_title': 'REGULATIONS made by the Sri ...,REGULATIONS made by the Sri Lanka Tea Board un...,2021-09-17,L.D.B. 2/1976(iii) / 2245/29
...,...,...,...,...,...,...,...,...
162,al-mqs-08 2008 04 24.pdf,# SRI LANKA TEA BOARD\n\n# Circular No.: AL/MQ...,# SRI LANKA TEA BOARD\n\n# Circular No.: AL/MQ...,"{\n ""parsed_title"": ""SRI LANKA TEA BOARD"",\n ...","{'parsed_title': 'SRI LANKA TEA BOARD', 'parse...",SRI LANKA TEA BOARD,24th April 2008,AL/MQS/08
163,guidelines_on_covid-19_in_tea_plantations_eng_...,# TEA RESEARCH INSTITUTE OF SRI LANKA\n\nIssue...,# TEA RESEARCH INSTITUTE OF SRI LANKA\n\nIssue...,"{\n ""parsed_title"": ""# GUIDELINES ON PREVENTI...",{'parsed_title': '# GUIDELINES ON PREVENTION A...,# GUIDELINES ON PREVENTION AND CONTROL OF SPRE...,May 2020,Guideline No: 02/2020
164,mf bl 119.pdf,# Sri Lanka Tea Board\n\n# Tea Commissioner’s ...,# Sri Lanka Tea Board\n\n# Tea Commissioner’s ...,"{\n ""parsed_title"": ""DELAY IN FURNISHING MONT...",{'parsed_title': 'DELAY IN FURNISHING MONTHLY ...,DELAY IN FURNISHING MONTHLY PRODUCTION RETURNS...,23rd December 1992,Circular No: MF / BL. 119
165,scan0002.pdf,# TKANSLATION\n\n# REGISTERED POST\n\n# SRI LA...,# TKANSLATION\n\n# REGISTERED POST\n\n# SRI LA...,"{\n ""parsed_title"": ""INTRODUCTION OF POISONOU...",{'parsed_title': 'INTRODUCTION OF POISONOUS SU...,INTRODUCTION OF POISONOUS SUBSTANCES TO TEA,"10th January, 1986",574


In [ ]:
merged_df = merged_df.drop(columns=['metadata_returned','parsed_metadata'])
merged_df.head()

,file_name,markdown_content_long,markdown_content_short,parsed_title,parsed_issue_date,parsed_reference_number
0,01_Sri-Lanka-Tea-Board-Standards-Directives-fo...,# MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...,# MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...,Sri Lanka Tea Board Minimum Quality Standards ...,"1st January, 2021",Circular No: AL/MQS-02/2021
1,07_General-Instruction-for-the-Submission-of-T...,# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...,# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...,GENERAL INSTRUCTION FOR THE SUBMISSION OF TEA ...,24.06.2013,AL/GI
2,1998 10 30.pdf,"# Sri Lanka Tea Board\n\n574, Galle Road, Colo...","# Sri Lanka Tea Board\n\n574, Galle Road, Colo...",ALL REGISTERED MANUFACTURERS OF TEA,30th October 1998,Circular of even number dated 22nd November 1993
3,2158-03_E.pdf,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,CONSTITUTION OF THE DEMOCRATIC SOCIALIST REPUB...,"09th January, 2020","No. 2158/3, 01- 649"
4,2245-29_E.pdf,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,# PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...,REGULATIONS made by the Sri Lanka Tea Board un...,2021-09-17,L.D.B. 2/1976(iii) / 2245/29


In [ ]:
import csv
# Sample function to save the DataFrame to CSV with markdown preservation
def save_dataframe_to_csv(df, filename):
    df.to_csv(filename,
              index=False,  # Do not write row indices
              quoting=csv.QUOTE_MINIMAL,  # Quote fields that contain special characters
              quotechar='"',  # Use double quotes to wrap fields
              escapechar='\\',  # Escape any quotes inside markdown content
              lineterminator='\n',  # Make sure to preserve newlines in markdown content
              encoding='utf-8')  # Ensure proper encoding to handle special characters in markdown



# Save the DataFrame to a CSV file with markdown content preserved
save_dataframe_to_csv(merged_df, '/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/llamaparse/llama_parsed_metadata_instant.csv')


# append metadata results to main csv

In [6]:
merged_df = pd.read_csv('/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/llamaparse/llama_parsed_metadata_instant.csv')

In [7]:
# Rename columns in the DataFrame
merged_df = merged_df.rename(columns={
    'file_name':'filename',
    'markdown_content_long': 'markdown_content',
    'parsed_title': 'llama_title',
    'parsed_issue_date': 'llama_issue_date',
    'parsed_reference_number': 'llama_reference_number'
})

In [8]:
merged_df.columns

Index(['filename', 'markdown_content', 'markdown_content_short', 'llama_title',
       'llama_issue_date', 'llama_reference_number'],
      dtype='object')

In [9]:
def append_merged_df_to_all_data(all_data, merged_df):
  """
  Appends the merged_df columns based on the 'file_name' to the all_data DataFrame.
  """
  # all_data = all_data.merge(merged_df[['file_name', 'markdown_content_long', 'parsed_title','parsed_issue_date', 'parsed_reference_number']], on='file_name', how='left')
  all_data = all_data.merge(merged_df[['filename', 'markdown_content', 'llama_title','llama_issue_date', 'llama_reference_number']], on='filename', how='left')

  return all_data


# Append merged_df columns to all_data
all_data = append_merged_df_to_all_data(all_data, merged_df)

# Print to confirm
all_data.head()

,id,class,filename,path,url,language_label,required_ocr,has_tables,text_path,quality,data_origin,retrieved_date_of_issuance,issuing_authority,retrieved_topic,text_content,PDF_or_text,markdown_content,llama_title,llama_issue_date,llama_reference_number
0,0,ACT,ACT_1.txt,,https://www.lawnet.gov.lk/tea-subsidy-3/,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Subsidy – LawNet,"Tea Subsidy\nActs Nos. 12 of 1958, 66 of 1961....",Text,"# Tea Subsidy\n\nActs Nos. 12 of 1958, 66 of 1...","Tea Subsidy Act, No. 12 of 1958","19th September, 1958","12 of 1958, 66 of 1961, 33 of 1966"
1,1,ACT,ACT_2.txt,,https://www.lawnet.gov.lk/tea-small-holdings-d...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Small Holdings Development (Amendment) – L...,|\n\n**Tea Small Holdings Development (Amendme...,Text,# Tea Small Holdings Development (Amendment)\n...,Tea Small Holdings Development (Amendment),"12th August, 1997",No. 21 of 1997
2,2,ACT,ACT_3.txt,,https://www.lawnet.gov.lk/tea-small-holdings-d...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Small Holdings Development (Amendment) – L...,|\n\n**Tea Small Holdings Development (Amendme...,Text,# Tea Small Holdings Development (Amendment)\n...,Tea Small Holdings Development (Amendment),"22nd October, 2003",Act No. 34 of 2003
3,3,ACT,ACT_4.txt,,https://www.lawnet.gov.lk/tea-small-holdings-d...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Small Holdings Development Law – LawNet,Tea Small Holdings Development Law\nA LAW TO E...,Text,# Tea Small Holdings Development Law\n\nA LAW ...,Tea Small Holdings Development Law,1975,35
4,4,ACT,ACT_5.txt,,https://www.lawnet.gov.lk/tea-research-board-a...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Research Board (Amendment) – LawNet,|\n\n**Tea Research Board (Amendment)** |\n\n|...,Text,# Tea Research Board (Amendment)\n\n# AN ACT T...,"AN ACT TO AMEND THE TEA RESEARCH BOARD ACT, NO...","6th November, 2006",43 of 2006


# remove copyright sections

In [12]:
import re

def clean_text_from_copyright(text):
    """
    Clean and preprocess the extracted text content to make it suitable for metadata extraction.
    This will remove unwanted text such as 'Copyright' and any excessive spaces or line breaks.
    """
    if not isinstance(text, str):
        return None

    # Remove specific unwanted text like 'Copyright' (case-insensitive) and everything after it
    text = re.sub(r'copyright[\s\S]*?(\n|\r|\Z)', '', text, flags=re.IGNORECASE)  # Remove copyright section and text after it

    return text

all_data['markdown_content'] = all_data['markdown_content'].apply(clean_text_from_copyright)

# get rows with NO_CONTENT_HERE and remove from dataset

In [ ]:
# # prompt: get markdown content rows where only NO CONTENT HERE exists and remove the rows

# def remove_no_content_rows(df):
#   """Removes rows from a DataFrame where 'markdown_content_short' contains only 'NO CONTENT HERE'."""
#   df = df[~df['markdown_content'].str.contains(r'^NO_CONTENT_HERE$', na=False)]
#   return df

# # TODO add code to print filename for these
# # Assuming 'merged_df' is the DataFrame with the 'markdown_content_short' column
# all_data = remove_no_content_rows(all_data)

# # Now 'all_data' has rows with only 'NO CONTENT HERE' removed.

In [13]:
import pandas as pd



def remove_no_content_rows(df):
    """Removes rows from a DataFrame where 'markdown_content' contains only 'NO_CONTENT_HERE'."""

    # First, identify rows where 'markdown_content' contains only 'NO CONTENT HERE'
    empty_rows = df[df['markdown_content'].str.contains(r'^NO_CONTENT_HERE$', na=False)]

    # Print filenames for these rows
    print("Rows removed (filename with NO_CONTENT_HERE):")
    for filename in empty_rows['filename']:
        print(filename)

    # Remove rows with 'NO CONTENT HERE'
    df = df[df['markdown_content'] != 'NO_CONTENT_HERE']

    # Add 'is_empty' column, marking rows where the content was 'NO CONTENT HERE'
    df['is_empty'] = df['markdown_content'] == 'NO_CONTENT_HERE'

    return df

# Call the function to remove rows with 'NO CONTENT HERE' and add the 'is_empty' column
all_data = remove_no_content_rows(all_data)

Rows removed (filename with NO_CONTENT_HERE):


In [ ]:
all_data = all_data.drop(columns=['is_empty'])
all_data.columns

Index(['id', 'class', 'filename', 'path', 'url', 'language_label',
       'required_ocr', 'has_tables', 'text_path', 'quality', 'data_origin',
       'retrieved_date_of_issuance', 'issuing_authority', 'retrieved_topic',
       'text_content', 'PDF_or_text', 'markdown_content', 'llama_title',
       'llama_issue_date', 'llama_reference_number'],
      dtype='object')

# detect duplicates

In [14]:
import pandas as pd
import re

# Function to normalize markdown content
def normalize_markdown(text):
    # Normalize: Convert to lowercase, remove extra whitespace, etc.
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.strip()  # Remove leading/trailing whitespace
    return text

# Apply normalization to the 'markdown_content' column
all_data['normalized_content'] = all_data['markdown_content'].apply(normalize_markdown)

# Mark duplicates based on the normalized content, keeping the first instance and marking all others as duplicates
all_data['is_duplicate'] = all_data.duplicated(subset='normalized_content', keep='first')

In [15]:
# Show rows where 'is_duplicate' is True
duplicates = all_data[all_data['is_duplicate'] == True]
# Display the result
print(duplicates)


      id      class                               filename  \
150  150  Circulers                OR1-51-1 2010 07 05.pdf   
153  153  Circulers                  OR3-4-1 1987 06 8.pdf   
167  167  Circulers  AL- MQS- REV -2010 - 1 2010 04 26.pdf   

                                                  path url language_label  \
150  /content/drive/MyDrive/Omdena_Challenge/new_LK...                 en   
153  /content/drive/MyDrive/Omdena_Challenge/new_LK...                 en   
167  /content/drive/MyDrive/Omdena_Challenge/new_LK...                 en   

    required_ocr has_tables text_path quality  ... retrieved_topic  \
150          yes         no              good  ...                   
153          yes         no              good  ...                   
167          yes        yes               bad  ...                   

                                          text_content PDF_or_text  \
150  8 @ceh) 6) ODIDEG\nGonmag Coonve Fonu\nSRI LAN...         PDF   
153  Any” aaa sey\nSR

In [16]:
# Create a new DataFrame with only non-duplicates
all_data = all_data[all_data['is_duplicate'] == False].copy()
all_data = all_data.drop(columns=['is_duplicate','normalized_content'])
print("\nNew DataFrame with non-duplicate rows:")
all_data.head()


New DataFrame with non-duplicate rows:


,id,class,filename,path,url,language_label,required_ocr,has_tables,text_path,quality,...,retrieved_date_of_issuance,issuing_authority,retrieved_topic,text_content,PDF_or_text,markdown_content,llama_title,llama_issue_date,llama_reference_number,is_empty
0,0,ACT,ACT_1.txt,,https://www.lawnet.gov.lk/tea-subsidy-3/,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,...,,Parliament of Sri Lanka ACT,Tea Subsidy – LawNet,"Tea Subsidy\nActs Nos. 12 of 1958, 66 of 1961....",Text,"# Tea Subsidy\n\nActs Nos. 12 of 1958, 66 of 1...","Tea Subsidy Act, No. 12 of 1958","19th September, 1958","12 of 1958, 66 of 1961, 33 of 1966",False
1,1,ACT,ACT_2.txt,,https://www.lawnet.gov.lk/tea-small-holdings-d...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,...,,Parliament of Sri Lanka ACT,Tea Small Holdings Development (Amendment) – L...,|\n\n**Tea Small Holdings Development (Amendme...,Text,# Tea Small Holdings Development (Amendment)\n...,Tea Small Holdings Development (Amendment),"12th August, 1997",No. 21 of 1997,False
2,2,ACT,ACT_3.txt,,https://www.lawnet.gov.lk/tea-small-holdings-d...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,...,,Parliament of Sri Lanka ACT,Tea Small Holdings Development (Amendment) – L...,|\n\n**Tea Small Holdings Development (Amendme...,Text,# Tea Small Holdings Development (Amendment)\n...,Tea Small Holdings Development (Amendment),"22nd October, 2003",Act No. 34 of 2003,False
3,3,ACT,ACT_4.txt,,https://www.lawnet.gov.lk/tea-small-holdings-d...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,...,,Parliament of Sri Lanka ACT,Tea Small Holdings Development Law – LawNet,Tea Small Holdings Development Law\nA LAW TO E...,Text,# Tea Small Holdings Development Law\n\nA LAW ...,Tea Small Holdings Development Law,1975,35,False
4,4,ACT,ACT_5.txt,,https://www.lawnet.gov.lk/tea-research-board-a...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,...,,Parliament of Sri Lanka ACT,Tea Research Board (Amendment) – LawNet,|\n\n**Tea Research Board (Amendment)** |\n\n|...,Text,# Tea Research Board (Amendment)\n\n# AN ACT T...,"AN ACT TO AMEND THE TEA RESEARCH BOARD ACT, NO...","6th November, 2006",43 of 2006,False


In [17]:
len(all_data)

167

# save markdown_content_long to md files in file directory for RAG pipeline

In [18]:
import pandas as pd
import os

# Function to save markdown content to a file
def save_markdown(content, markdown_path):
    with open(markdown_path, 'w') as f:
        f.write(content)

# Function to process each row and save markdown
def process_row(row):
    # Extract filename and path
    pdf_filename = row['filename']  # e.g., 'file1.pdf'

    # Strip '.pdf' or '.txt' extension and replace with '.md'
    filename_without_extension = os.path.splitext(pdf_filename)[0]  # e.g., 'file1'
    markdown_filename = f"{filename_without_extension}.md"          # e.g., 'file1.md'

    # Determine the source file path for markdown
    if row['PDF_or_text'] == 'PDF':
        markdown_path = os.path.join(os.path.dirname(row['path']), markdown_filename)  # Use path for PDF
        markdown_content = row['markdown_content']  # Directly use markdown_content for PDF
    elif row['PDF_or_text'] == 'Text':
        markdown_path = os.path.join(os.path.dirname(row['text_path']), markdown_filename)  # Use text_path for Text
        markdown_content = row['markdown_content']  # Directly use markdown_content for Text (already extracted)

    # Save the markdown content to the file
    save_markdown(markdown_content, markdown_path)

    # Return the constructed markdown path to be used in the DataFrame
    return markdown_path


# Apply the process_row function to each row and create a new 'markdown_path' column
all_data['markdown_path'] = all_data.apply(process_row, axis=1)

# Print the updated DataFrame with markdown paths
all_data.head()


,id,class,filename,path,url,language_label,required_ocr,has_tables,text_path,quality,...,issuing_authority,retrieved_topic,text_content,PDF_or_text,markdown_content,llama_title,llama_issue_date,llama_reference_number,is_empty,markdown_path
0,0,ACT,ACT_1.txt,,https://www.lawnet.gov.lk/tea-subsidy-3/,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,...,Parliament of Sri Lanka ACT,Tea Subsidy – LawNet,"Tea Subsidy\nActs Nos. 12 of 1958, 66 of 1961....",Text,"# Tea Subsidy\n\nActs Nos. 12 of 1958, 66 of 1...","Tea Subsidy Act, No. 12 of 1958","19th September, 1958","12 of 1958, 66 of 1961, 33 of 1966",False,/content/drive/MyDrive/Omdena_Challenge/new_LK...
1,1,ACT,ACT_2.txt,,https://www.lawnet.gov.lk/tea-small-holdings-d...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,...,Parliament of Sri Lanka ACT,Tea Small Holdings Development (Amendment) – L...,|\n\n**Tea Small Holdings Development (Amendme...,Text,# Tea Small Holdings Development (Amendment)\n...,Tea Small Holdings Development (Amendment),"12th August, 1997",No. 21 of 1997,False,/content/drive/MyDrive/Omdena_Challenge/new_LK...
2,2,ACT,ACT_3.txt,,https://www.lawnet.gov.lk/tea-small-holdings-d...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,...,Parliament of Sri Lanka ACT,Tea Small Holdings Development (Amendment) – L...,|\n\n**Tea Small Holdings Development (Amendme...,Text,# Tea Small Holdings Development (Amendment)\n...,Tea Small Holdings Development (Amendment),"22nd October, 2003",Act No. 34 of 2003,False,/content/drive/MyDrive/Omdena_Challenge/new_LK...
3,3,ACT,ACT_4.txt,,https://www.lawnet.gov.lk/tea-small-holdings-d...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,...,Parliament of Sri Lanka ACT,Tea Small Holdings Development Law – LawNet,Tea Small Holdings Development Law\nA LAW TO E...,Text,# Tea Small Holdings Development Law\n\nA LAW ...,Tea Small Holdings Development Law,1975,35,False,/content/drive/MyDrive/Omdena_Challenge/new_LK...
4,4,ACT,ACT_5.txt,,https://www.lawnet.gov.lk/tea-research-board-a...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,...,Parliament of Sri Lanka ACT,Tea Research Board (Amendment) – LawNet,|\n\n**Tea Research Board (Amendment)** |\n\n|...,Text,# Tea Research Board (Amendment)\n\n# AN ACT T...,"AN ACT TO AMEND THE TEA RESEARCH BOARD ACT, NO...","6th November, 2006",43 of 2006,False,/content/drive/MyDrive/Omdena_Challenge/new_LK...


# save main csv

In [20]:
import csv

# Sample function to save the DataFrame to CSV with markdown preservation
def save_dataframe_to_csv(df, filename):
    df.to_csv(filename,
              index=False,  # Do not write row indices
              quoting=csv.QUOTE_MINIMAL,  # Quote fields that contain special characters
              quotechar='"',  # Use double quotes to wrap fields
              escapechar='\\',  # Escape any quotes inside markdown content
              lineterminator='\n',  # Make sure to preserve newlines in markdown content
              encoding='utf-8')  # Ensure proper encoding to handle special characters in markdown



# Save the DataFrame to a CSV file with markdown content preserved
save_dataframe_to_csv(all_data, '/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/llamaparse/v2_new_LK_tea_dataset_updated.csv')


# estimating number of token from markdown file

In [21]:
import pandas as pd

# Function to count words and characters in text
def count_words_and_characters(text):
    words = text.split()  # Split by whitespace to get words
    num_words = len(words)
    num_characters = len(text)
    return num_words, num_characters

# Function to count words and characters for the 'text_content' column in the DataFrame
def count_stats_in_dataframe(df):
    total_words = 0
    total_characters = 0

    # Iterate over the 'text_content' column in the DataFrame
    for text in df['markdown_content'].dropna():  # Drop any NaN values in text_content
        num_words, num_characters = count_words_and_characters(text)
        total_words += num_words
        total_characters += num_characters

    return total_words, total_characters

# Main script
if __name__ == "__main__":

    # Count words and characters
    total_words, total_characters = count_stats_in_dataframe(all_data)

    # Output the results
    print("\nTotal Stats for 'markdown_content' column in DataFrame:")
    print(f"Total Words: {total_words}")
    print(f"Total Characters: {total_characters}")



Total Stats for 'markdown_content' column in DataFrame:
Total Words: 181510
Total Characters: 1122492
